In [22]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split data into source and target domains
X_source, X_target, y_source, y_target = train_test_split(X, y, test_size=0.5, random_state=42)

# Convert numpy arrays to PyTorch tensors
X_source = torch.from_numpy(X_source).float()
X_target = torch.from_numpy(X_target).float()
y_source = torch.from_numpy(y_source).long()

# Define the feature extractor
class FeatureExtractor(nn.Module):
    def __init__(self, input_size):
        super(FeatureExtractor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

# Define the task-specific network
class TaskNetwork(nn.Module):
    def __init__(self,input_size):
        super(TaskNetwork, self).__init__()
        self.feature_extractor = FeatureExtractor(input_size)
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 3)  # Output layer for Iris dataset with 3 classes
    
    def forward(self, x):
        features = self.feature_extractor(x)
        x = F.relu(self.fc1(features))
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Initialize the models
task_network = TaskNetwork(input_size=4)

# Freeze the parameters of the pre-trained feature extractor
for param in task_network.feature_extractor.parameters():
    param.requires_grad = False

# Define optimizer for the task-specific network only
optimizer_task = optim.Adam(task_network.parameters(), lr=0.001)

# Training loop
epochs = 500
for epoch in range(epochs):
    # Train task network
    task_network.train()  # Set the task network to training mode
    optimizer_task.zero_grad()
    source_features = task_network.feature_extractor(X_source)
    predicted = task_network(X_source)
    loss_task = F.cross_entropy(predicted, y_source)
    loss_task.backward()
    optimizer_task.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{epochs}], Task Network Loss: {loss_task.item()}')

# Evaluate the model (similar to previous evaluation code)

Epoch [0/500], Task Network Loss: 1.1001805067062378
Epoch [10/500], Task Network Loss: 1.0939676761627197
Epoch [20/500], Task Network Loss: 1.0903115272521973
Epoch [30/500], Task Network Loss: 1.087016224861145
Epoch [40/500], Task Network Loss: 1.0831530094146729
Epoch [50/500], Task Network Loss: 1.078602910041809
Epoch [60/500], Task Network Loss: 1.073197841644287
Epoch [70/500], Task Network Loss: 1.0667837858200073
Epoch [80/500], Task Network Loss: 1.0591986179351807
Epoch [90/500], Task Network Loss: 1.0502936840057373
Epoch [100/500], Task Network Loss: 1.0399631261825562
Epoch [110/500], Task Network Loss: 1.0281555652618408
Epoch [120/500], Task Network Loss: 1.0149377584457397
Epoch [130/500], Task Network Loss: 1.0005426406860352
Epoch [140/500], Task Network Loss: 0.985356867313385
Epoch [150/500], Task Network Loss: 0.9699095487594604
Epoch [160/500], Task Network Loss: 0.9547646045684814
Epoch [170/500], Task Network Loss: 0.9404261112213135
Epoch [180/500], Task Net

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Evaluation function
def evaluate_model(X_test, y_test, model):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient tracking
        y_pred_probs = model(X_test)
        _, y_pred = torch.max(y_pred_probs, 1)  # Convert probabilities to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test.numpy(), y_pred.numpy())
    precision = precision_score(y_test.numpy(), y_pred.numpy(), average='macro')
    recall = recall_score(y_test.numpy(), y_pred.numpy(), average='macro')
    f1 = f1_score(y_test.numpy(), y_pred.numpy(), average='macro')

    # Calculate confusion matrix
    cm = confusion_matrix(y_test.numpy(), y_pred.numpy())

    return accuracy, precision, recall, f1, cm

# Splitting the test data
X_source_test, X_target_test, y_source_test, y_target_test = train_test_split(X_target, y_target, test_size=0.5, random_state=42)

# Evaluate the model
accuracy, precision, recall, f1, cm = evaluate_model(X_source_test, torch.from_numpy(y_source_test).long(), task_network)

# Print evaluation results
print("Evaluation Results:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)


Evaluation Results:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[12  0  0]
 [ 0 12  0]
 [ 0  0 13]]
